# Math 164 Project 1 Report - Option Pricing

### We want to analyze the dynamics of underlying security price. 
### We will use the Black-Scholes Formula to estimate the European call/put option price with historical volatility and implied volatility respectively.

## Part I. Black-Scholes Formula

The Black-Scholes formula is a simple way to estimate the option price with the given maturity ($T$), strike price ($K$), current stock price ($S_0$), interest rate (risk-free rate) ($r$), dividend rate ($\delta$) and volatility ($\sigma^2$). In this project, we assume divident rate, $\delta=0$ for simplicity.

Thus, we will use the following Black-Scholes formulas in this project to estimate option prices:

Call Option:
\begin{equation}
C=S_0N(d_1)-Ke^{-rT}N(d_2)
\end{equation}

Put Option:
\begin{equation}
P=Ke^{-rT}N(-d_2)-S_0N(-d_1)
\end{equation}

where
\begin{equation}
d_1=\frac{ln(\frac{S_0}{K})+(r+\frac{1}{2}\sigma^2)T}{\sigma\sqrt{T}}\\
d_2=d_1-\sigma\sqrt{T}
\end{equation}

From Yahoo Finance, we can easily obtain all of the required information except volatility. Thus, the goal of this project is to assess the accuracy of option price estimation using various volatility estimating methods.

## Part II. Preparation

We will be using the data of options of Apple Inc traded on March 20, 2018 for this project. We obtain this dataset from Yahoo Finance.

From treasury.gov, we use the value of Treasury 20-Yr CMT on March 20, 2018 as the interest rate. i.e. $r=3.01\%$

### Preparation

In [3]:
import numpy as np
import scipy.stats
from math import *
import csv
from datetime import datetime
import matplotlib.pyplot as plt
import sys
from scipy.optimize import curve_fit
import statistics
import time
from sklearn.svm import SVR
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import learning_curve
from sklearn.kernel_ridge import KernelRidge

To start the project, we import all dependencies needed. We will be using sci-kit learn to approximate graphs in Part IV. Implied Volatility.

In [4]:
### GLOBAL VARIABLES ###
# We set risk-free rate at 3.01%. Data obtained from treasury.gov, using Treasury 20-yr CMT
RATE = 0.0301
DAY = 365
BEST = 1
DATE = '0-0-0'
DATE = '3/20/2018'   ### DEBUG ###
PT_NUM = 10

We introduce some global variables. Clearly, the risk free rate is constant throughout. We set the total days in a year as 365. We can also use 261, which is the total number of calendar working days in 2018. This number will not have a noticeable influence to our result. DATE is initialized for convenience. PT_NUM is a parameter indicating the minimum number of points required for Implied Volatlity estimation in Part IV.

In [6]:
# Load historical option data on 2018/3/2
with open("../data/AAPL_032018.csv") as data:
    data_reader = csv.reader(data)
    raw_data = list(data_reader)

# Load historical price
with open("../data/AAPL_HP.csv") as data:
    data_reader = csv.reader(data)
    raw_hp = list(data_reader)

AAPL_032018.csv stores the details of options traded on 3/22/2018. 

AAPL_HP.csv stores the historical stock prices of Apple Inc. We import these 2 tables.

## Part III. Historical Volatility

In this section, we assume volatility is constant throughout the day for all options. We will estimate the volatility on March 20, 2018 by learning the historical stock prices of Apple Inc. 

We use MLE to calculate the volatility of the day:
\begin{equation}
\bar{\sigma}^2=\frac{1}{n}\sum_{t=1}^n(R_t-\bar{R})^2
\end{equation}
where
\begin{equation}
R_t=log(\frac{S_t}{S_{t-1}})
\end{equation}

### Estimate Historical Volatility (HV)

In [9]:
# Calculate average R_t in the given data
def average_volatility(raw, start=1, end=0):
    # parameter:
        # raw - matrix of history price
    r = len(raw)
    count = 0
    # print("Number of records = " + str(r))
    sum = 0
    for i in range (start + 1, r):
        # if i <= start:
        #     continue
        if end != 0 and i > end:
            break
        p1 = float(raw[i-1][4])
        p2 = float(raw[i][4])
        temp1 = log(p2/p1)
        # print(temp1)
        sum = sum + temp1
        count = count + 1
    avg = sum/count
    # print("Sum is " + str(sum))
    # print("Avg is " + str(avg))
    return avg

Function average_volatility will return $\bar{R}$ which is required to calculate historical volatility.

In [8]:
# Calculate estimated variance - volatility
def get_volatility(raw, start=1, end=0):
    # parameter:
        # raw - matrix of history price (including header)
    r = len(raw)
    sum = 0
    count = 0
    avg = average_volatility(raw, start=start, end=end)
    for i in range(start+1, r):
        if end != 0 and i > end:
            break
        p1 = float(raw[i-1][4])
        p2 = float(raw[i][4])
        temp1 = log(p2/p1)
        sum = sum + temp1 * temp1
        count = count + 1

    # Use variance formula which \sum (X-E[X])^2 = E[X^2] - E[X]^2
    E_X2 = sum/count
    var = E_X2 - avg * avg
    # We times volatility of one business day with #business days
    var = 260 * var
    print("Estimated volatility is " + str(var))
    return var

Function get_volatility will return $\bar{\sigma}^2$, which is the historical volatility (HV) that we want.

### Evaluation of HV

After obtaining HV, we want to assess its accuracy in predicting option prices. We will plug in the calculated HV to the Black-Scholes Formula to obtain the estimated put/call option prices. We will compare these estimated results with the real results to check the accuracy of HV.

In [12]:
def get_maturity(raw_trade, raw_expire):
    # parameter:
        # raw_trade - trade date of the option
        # raw_expire - expiration date of the option
    # print("GET_MATURITY CALLED", raw_trade, raw_expire)
    trade_date = datetime.strptime(raw_trade, '%m/%d/%Y')
    trade_days = float(datetime.strftime(trade_date, '%j')) * DAY / 365
    trade_year = float(datetime.strftime(trade_date, '%Y'))
    expire_date = datetime.strptime(raw_expire, '%m/%d/%Y')
    expire_days = float(datetime.strftime(expire_date, '%j')) * DAY / 365
    expire_year = float(datetime.strftime(expire_date, '%Y'))
    expire_days = expire_days + (expire_year - trade_year) * DAY
    day_difference = expire_days - trade_days
    h = day_difference / DAY        # Length of maturity
    return h

Function get_maturity will return maturity in number of years.

In [13]:
# Calculate d1 and d2
def get_d1_d2(sd, h, s, k):
    # parameter:
        # sd - sqrt(Volatility)
        # h - Maturity
        # s - Stock Price, S_0
        # k - Strike Price, K
    d1 = (1/(sd * sqrt(h))) * ((log(s/k)) + (RATE + ((sd * sd)/2)) * h)
    # Use d2 = d1 - sqrt(vol) * sqrt(maturity)
    d2 = d1 - ( sd * sqrt(h) )
    # print(d1, d2)
    return d1, d2

Function get_d1_d2 will return values of d1 and d2 for Black-Scholes formula.

In [14]:
# Estimate option price using BS formula
def bs_call_put(h, s, k, sd):
    # parameter:
        # h - Maturity
        # s - Stock Price, S_0
        # k - Strike Price, K
        # sd - sqrt(Volatility)
    d1, d2 = get_d1_d2(sd, h, s, k)
    # We ignore divident here
    N1 = scipy.stats.norm.cdf(d1)
    N2 = scipy.stats.norm.cdf(d2)
    call = (N1 * s) - (N2 * k * exp(- RATE * h))
    N_1 = 1 - N1
    N_2 = 1 - N2
    put = (N_2 * k * exp(- RATE * h)) - (s * N_1)
    return call, put

Function bs_call_put will return call and put option prices with the given parameters.

In [15]:
# Calculate estimated call option price
def estimate_call_put(raw, vol, start=1, end=0):
    # parameters:
        # raw - raw_data (including header)
        # vol - estimated volatility
        # start - starting row
        # end - ending row
    r = len(raw)
    # Calculate Standard Deviation
    sd = sqrt(vol)
    # print("Standard Deviation is " + str(sd))
    # Obtain maturity of option

    if sd == 0:
        # If volatility is 0, call/put options will have price of 0 
        # since future prices of stock price is fully predicatable with no risk.
        return [[0,0]]*r

    output = []

    for i in range(r):
        # Get T-t
        if i < start:
            continue
        if i >= end and end != 0:
            break
        raw_trade = raw[i][3]
        raw_expire = raw[i][10]
        year_difference = get_maturity(raw_trade, raw_expire)
        # print("Maturity is " + str(year_difference))

        # Get Initial Stock price
        S = float(raw[i][26])
        # print("Stock price is " + str(S))

        # Get Strike price
        K = float(raw[i][11])
        # print("Strike price is " + str(K))

        # Get call, put option premium
        call, put = bs_call_put(year_difference, S, K, sd)
        output.append([call, put])
        # print((call, put))
        
    return output

Function estimate_call_put will take in the raw option dataset and the HV to estimate option prices. It will return a list of estimated calls and puts prices.

In [16]:
# Evaluate how accurate my estimation is
def check_estimation(est, raw, start=1, end=0, plot=0):
    # parameters:
        # est - estimated call & put prices
        # raw - raw_data (including header)
        # start - starting row
        # end - ending row
    r = len(raw)
    diff_table = []
    perc_table = []
    est_l = []
    act_l = []
    for i in range(start, r):
        if i >= end and end != 0:
            break
        act_i = float(raw[i][5])
        act_l.append(act_i)
        if raw[i][12] == 'C':
            est_i = float(est[i-start][0])
            abs_diff = abs(act_i-est_i)
            diff_table.append(abs_diff)
            perc_table.append(abs_diff/act_i)
            est_l.append(est_i)
        elif raw[i][12] == 'P':
            est_i = float(est[i-start][1])
            abs_diff = abs(est_i - act_i)
            diff_table.append(abs_diff)
            perc_table.append(abs_diff/act_i)
            est_l.append(est_i)
        else:
            print("Neither C or P detected at i = " + str(i))

    # Print average absolute difference and percentage difference
    num = 0
    if (end == 0):
        num = r - start
    else:
        num = end - start
    avg_abs_diff = sum(diff_table)/num
    avg_perc_diff = sum(perc_table)/num
    print("Average absolute error is " + str(avg_abs_diff))
    print("Average percentage error is " + str(avg_perc_diff))
    global BEST
    if BEST > avg_perc_diff:
        BEST = avg_perc_diff
    print("Best result: " + str(BEST))

    # Plot Dots vs Linear
    if plot:
        plt.scatter(est_l, act_l)
        plt.plot([1],[1])
        plt.xlabel('estimated data')
        plt.ylabel('actual data')
        plt.show()

    return avg_abs_diff, avg_perc_diff

Function check_estimation will tell us the average absolute difference and average percentage difference of our estimated option prices using HV. We can also plot a graph showing the distribution of estimated option prices against the actual prices in scatter.

In [ ]:
report = [['First Period', 'last Period', 'Risk Free Rate', 'Estimated Volatility', 'Avg Abs Err', 'Avg Perc Err']]

len_hp = len(raw_hp)
rate = RATE

for i in range(1, len_hp, 20):
    k = 2045   # fix to the last day
    # print(i, k)
    # Estimate volatility using data from i to k
    print("Using rate = " + str(RATE))
    print("Using data from " + raw_hp[i][0] + " to " + raw_hp[k][0])
    volatility = get_volatility(raw_hp, start=i, end=k)
    call_put_table = estimate_call_put(raw_data, volatility)
    avg_abs_err, avg_perc_err = check_estimation(call_put_table, raw_data)
    report.append([raw_hp[i][0], raw_hp[k][0], RATE, volatility, avg_abs_err, avg_perc_err])

for i in range(1995, 2045):
    # k = len_hp - j - 1
    k = 2045   # fix to the last day
    # Estimate volatility using data from i to k
    print("Using rate = " + str(RATE))
    print("Using data from " + raw_hp[i][0] + " to " + raw_hp[k][0])
    volatility = get_volatility(raw_hp, start=i, end=k)
    call_put_table = estimate_call_put(raw_data, volatility)

    avg_abs_err, avg_perc_err = check_estimation(call_put_table, raw_data)
    report.append([raw_hp[i][0], raw_hp[k][0], RATE, volatility, avg_abs_err, avg_perc_err])

We check HV of different periods. The results of HV estimation and evaluation are stored in [../report/report_HV_assessment.csv](../report/report_HV_assessment.csv).

In [18]:
with open("../report/report.csv", 'w') as book:
            wr = csv.writer(book, delimiter=',', lineterminator='\n')
            for row in report:
                wr.writerow(row, )

Write the report generated above into a csv file. (The report has already been generated and renamed.)

### Concluson of Historical Volatility

Among all obtained HV, the best result gives average percentage error at 23.5%, which is not a satisfactory result. 

HV is not accurate as expected since the volatility is very volatile itself. It could vary every second. It is indeed stochastic. Thus, using historical data in the past days or months is not a good idea to approximate volatility.

## Part IV. Implied Volatility

As we have shown in Part III that volatility is so volatile that using data in a long period time in the past is not a good idea, we believe using data in a short time period that is very close to the trade time would give us a more accurate estimation of volatility, and thus, a more accurate estimation of option prices.